In [52]:
import pandas as pd
import json
import requests

In [53]:
print('\n\n\n=======================\nALGORITHMIC TRADING BOT\
    \nTEST ENVIRONMENT\n=======================\n\n\n')

balance = 10000
print(f'Starting Balance: ${balance}')

print('\nAlgorithm Rules:\nTrade Size = 5% of balance.\nTP/SL = 1 ATR\nDaily Profit Target = 2% Profit')




ALGORITHMIC TRADING BOT    
TEST ENVIRONMENT



Starting Balance: $10000

Algorithm Rules:
Trade Size = 5% of balance.
TP/SL = 1 ATR
Daily Profit Target = 2% Profit


In [54]:
# import datetime
starting_time = 1669852800000
limit = 1000
timeframe = 60 #in minutes

def convert_minutes_to_milliseconds(minutes):
    milliseconds = 60000 * minutes
    return milliseconds


url1 = f'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit={limit}&endTime={starting_time}'
url2 = f'https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit={limit}&endTime={starting_time - limit*(convert_minutes_to_milliseconds(timeframe))}'

urls = [url1, url2]

In [55]:
def get_bars(symbol):
    df = pd.DataFrame()
    for i in urls:
        data = json.loads(requests.get(i).text)
        print(i)
        for row in data:
            del row[5:]
        df = df.append(data)
    df.columns = ['date',
                'open', 'high', 'low', 'close']
    return df

eth_df_raw = get_bars('ETHUSDT')
eth_df_raw = eth_df_raw.sort_values(by='date')

https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1669852800000


/var/folders/2l/h2r8kh1x2jx7lbwdk0x41gp80000gn/T/ipykernel_5177/3063555294.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h&limit=1000&endTime=1666252800000


In [56]:
%%script echo skipping

eth_df_raw['open'] = pd.to_numeric(eth_df_raw['open'])
eth_df_raw['high'] = pd.to_numeric(eth_df_raw['high'])
eth_df_raw['low'] = pd.to_numeric(eth_df_raw['low'])
eth_df_raw['close'] = pd.to_numeric(eth_df_raw['close'])
eth_df_raw['date'] = pd.to_datetime(eth_df_raw['date'], unit = 'ms')

skipping


In [57]:
%%script echo skipping
eth_df_raw

,date,open,high,low,close
0,1662656400000,1630.23000000,1639.41000000,1620.20000000,1630.59000000
1,1662660000000,1630.59000000,1636.47000000,1625.30000000,1630.55000000
2,1662663600000,1630.55000000,1650.00000000,1624.32000000,1649.60000000
3,1662667200000,1649.60000000,1663.56000000,1639.83000000,1644.92000000
4,1662670800000,1644.91000000,1645.00000000,1630.00000000,1639.24000000
...,...,...,...,...,...
995,1669838400000,1287.20000000,1295.76000000,1285.00000000,1295.29000000
996,1669842000000,1295.29000000,1299.00000000,1287.73000000,1297.07000000
997,1669845600000,1297.07000000,1309.77000000,1292.24000000,1298.91000000
998,1669849200000,1298.90000000,1306.51000000,1290.84000000,1294.46000000


In [58]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    raw_df = raw_df.iloc[200:]

    raw_df = raw_df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    raw_df['trigger_long'] = np.where(((raw_df['macd'] > raw_df['signal']) & (raw_df['close'] > raw_df['200_ema']) & (raw_df['macd'] < 0) & (raw_df['signal'] > raw_df['macd'].shift())), 1, 0)

    raw_df['trigger_short'] = np.where(((raw_df['signal'] > raw_df['macd']) & (raw_df['close'] < raw_df['200_ema']) & (raw_df['macd'] > 0) & (raw_df['signal'] < raw_df['macd'].shift())), 1, 0)


    return raw_df